In [48]:
import pandas as pd
import math

In [49]:
data_set = pd.read_csv('book_ds.csv')

data_set = data_set.drop_duplicates(subset=['title', 'description'])

data_set = data_set.loc[:,
           ['title', 'description', 'language', 'genres', 'publishDate', 'awards', 'pages', 'publisher', 'isbn',
            'coverImg', 'author', 'price']]
data_set = data_set.reset_index(drop=True)

english_regex = r"[a-zA-Z0-9\s.,?!']+"
data_set = data_set[(data_set['language'] == "English") |
                    (data_set['language'].isna() & data_set['title'].str.fullmatch(english_regex)) &
                    (~data_set['description'].isna())]

data_set = data_set.rename(
    columns={'publishDate': 'publicationDate', 'author': 'authors', 'isbn': 'ISBN', 'coverImg': 'imageLink'})
data_set['language'] = data_set['language'].fillna('English')
data_set.dropna(axis=0, inplace=True)

data_set['available'] = 10

data_set['price'] = pd.to_numeric(data_set['price'], errors='coerce')
price_mean = data_set['price'].mean()
data_set['price'] = data_set['price'].fillna(price_mean)
data_set['price'] = data_set['price'].apply(lambda x: math.trunc(x))


def split_authors(authors_string):
    return [author.strip() for author in authors_string.split(',')]


data_set['authors'] = data_set['authors'].apply(split_authors)

data_set = data_set[~data_set['publicationDate'].astype(str).str.isalnum()]


data_set['publicationDate'] = pd.to_datetime(data_set['publicationDate'], errors='coerce')
data_set['publicationDate'] = data_set['publicationDate'].astype(str)
data_set = data_set.sample(n=5000, random_state=42)

/var/folders/vn/gx5pnsy51rxddtty65n5qbhm0000gn/T/ipykernel_88755/1009886183.py:47: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_set['publicationDate'] = pd.to_datetime(data_set['publicationDate'], errors='coerce')


,title,description,language,genres,publicationDate,awards,pages,publisher,ISBN,imageLink,authors,price,available
6578,Sunrise,Erin Hunter's #1 nationally bestselling Warrio...,English,"['Fantasy', 'Animals', 'Fiction', 'Middle Grad...",2009-04-21,[],318,HarperCollins,9780060892173,https://i.gr-assets.com/images/S/compressed.ph...,[Erin Hunter],2,10
16808,Night's Child,"Morgan's saga continues in this double-length,...",English,"['Young Adult', 'Fantasy', 'Witches', 'Paranor...",2003-08-25,[],320,Puffin,9780142501191,https://i.gr-assets.com/images/S/compressed.ph...,[Cate Tiernan],1,10
8387,Three Cups of Tea,This young readers edition of the worldwide be...,English,"['Nonfiction', 'Memoir', 'Biography', 'Educati...",2009-01-22,['Dayton Literary Peace Prize Nominee for Nonf...,240,Puffin Books,9780142414125,https://i.gr-assets.com/images/S/compressed.ph...,"[Sarah L. Thomson (Adapter), Greg Mortenson (G...",2,10
8202,Love Bites,True Love: Good From The First Bite...Etienne ...,English,"['Paranormal Romance', 'Romance', 'Vampires', ...",2004-01-06,[],373,Love Spell,9780505525536,https://i.gr-assets.com/images/S/compressed.ph...,[Lynsay Sands (Goodreads Author)],1,10
10184,The Owl Killers,"From the author of Company of Liars, hailed as...",English,"['Historical Fiction', 'Fiction', 'Historical'...",2009-06-01,['Shirley Jackson Award Nominee for Novel (200...,640,Michael Joseph,9780718153205,https://i.gr-assets.com/images/S/compressed.ph...,[Karen Maitland (Goodreads Author)],2,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49678,Second Sight,Financially straitened and on the path to spin...,English,"['Romance', 'Historical Romance', 'Paranormal'...",2007-03-27,[],338,Jove Books,9780515142808,https://i.gr-assets.com/images/S/compressed.ph...,"[Amanda Quick (Goodreads Author), Jayne Ann Kr...",3,10
4668,One Silent Night,"While the world carries on unawares, Stryker, ...",English,"['Paranormal Romance', 'Paranormal', 'Romance'...",2008-11-02,[],310,St. Martin's Press,9780312947064,https://i.gr-assets.com/images/S/compressed.ph...,[Sherrilyn Kenyon (Goodreads Author)],3,10
5787,The Long Earth,"From the back jacket:NORMALLY, WHEN THERE WAS ...",English,"['Science Fiction', 'Fantasy', 'Fiction', 'Sci...",2012-06-19,['Goodreads Choice Award for Science Fiction (...,336,HarperCollins Harper,9780062067753,https://i.gr-assets.com/images/S/compressed.ph...,"[Terry Pratchett, Stephen Baxter]",7,10
3135,The Tender Bar: A Memoir,In the grand tradition of landmark memoirs - a...,English,"['Memoir', 'Nonfiction', 'Biography', 'Autobio...",2006-08-01,['Audie Award for Narration by the Author or A...,416,Hyperion,9780786888764,https://i.gr-assets.com/images/S/compressed.ph...,[J.R. Moehringer],3,10


add to database

In [ ]:
import requests
post_url = "http://localhost:8080/books/"


def process_text(text):
    result =  ''.join(char for char in text if char.isalnum() or char == "," or char == ' ')
    return result.split(',')

def send_book_data(book_data):
    try:
        book_data['genres'] = process_text(book_data['genres'])
        book_data['awards'] = process_text(book_data['awards'])
        book_data['publicationDate'] = str(book_data['publicationDate'])
        book_data['isbn'] = book_data['ISBN']
        response = requests.post(post_url, json=book_data)
        response.raise_for_status()
        print(f"Book data sent successfully! Status code: {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"Error sending book data: {e}")


for index, row in data_set.iterrows():
    send_book_data(row.to_dict())